## Importation de donnnes

In [1]:
data<-read.csv("data_visco.csv")
data_lasso <- read.csv("data_lasso.csv")
data_AIC <- read.csv("data_AIC.csv")


In [2]:
dim(data_lasso)
dim(data_AIC)

[1] 7247   29

[1] 7247   20

## t-IMSE standard

In [3]:
# Charger la librairie
library("KrigInv") #Pour t-IMSE
library("nloptr") #Pour la fonction d'optimisation Cobyla

Le chargement a nécessité le package : DiceKriging



In [4]:
x = data[1:60,1:116]
#x = matrix(sample(5:500, 116*30, replace=TRUE),nrow = 30)
data_reduced = data[1:120,1:116]
#des = data_reduced[c(-1)]
shape = c(dim(data_reduced))
noise = matrix(runif(prod(shape),min = 0.00000001,max = 0.00000002), nrow = 120)
mod = km(formula = ~1, design = data_reduced + noise, response = data_reduced[1])
tmse_optim(x, model = mod, T = 400)


optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  12076 0.78 1.966423e-08 44.86 1.94

[1] 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021
 [9] 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021
[17] 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021
[25] 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021
[33] 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021
[41] 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021
[49] 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021 363.8021
[57] 363.8021 363.8021 363.8021 363.8021

In [5]:
fn<-function(x){
  tmse_optim(x, model = mod, T = 30)
}
contr<-function(x){
  h<-numeric(2)
  h[1]<- 100-2*x #g(x)>=0
  h[2]<- x  #-g(x)>=0
  return(h)
}

#cobyla(x[1,1:116],fn,hin=contr)

## Adaptation t-IMSE à nos contraintes

Au lieu d'utiliser la fonction **tmse_optim** standard, il nous faut considerer les contraintes du simplex

In [6]:
#simulation de x unifome sur [0,1] et la somme egale à 1
simul <- function(d)
{
  # set.seed(18)
  u = runif(n=d-1, min=0,max=1)
  #replicate(n=5,u) si on doit tester avec plusieurs vecteurs diff
  u = c(0,u,1)
  u = sort(u)
  x = u[2:(d+1)] - u[1:d]  
  return(x)
}

In [7]:
g = function (x, model, T, method.param = NULL) 
{
    y <- t(x)
    if (ncol(y) == model@d) 
        z <- y
    if (ncol(y) != model@d) 
        z <- x
    krig <- predict_nobias_km(object = model, newdata = as.data.frame(z), 
        type = "UK", se.compute = TRUE)
    mk <- krig$mean
    sk <- krig$sd
    if (is.null(method.param)) 
        method.param <- 0
    epsilon <- method.param
    if (length(T) == 1) {
        W <- 1/sqrt(2 * pi * (sk^2 + epsilon^2)) * exp(-0.5 * 
            ((mk - T)/sqrt(sk^2 + epsilon^2))^2)
    }
    else {
        W0 <- 1/sqrt(2 * pi * (sk^2 + epsilon^2))
        W <- 0
        for (i in 1:length(T)) {
            Ti <- T[i]
            W <- W + W0 * exp(-0.5 * ((mk - Ti)/sqrt(sk^2 + epsilon^2))^2)
        }
    }
    g <- W * sk^2
    return(g)
}

In [23]:
help(tmse_optim)

In [8]:
d = 29
N = 10*d
s = 0
data_Lasso_reduced = data_lasso[1:N,]
shape = c(dim(data_Lasso_reduced))
noise = matrix(runif(prod(shape),min = 0.0001,max = 0.0002), nrow = N)
mod = km(formula = ~1, design = data_Lasso_reduced+noise, response = data_Lasso_reduced$vis)
x = data_Lasso_reduced[1:50,]
tmse_optim(x, model = mod, T = 400)



optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  476715169234 0.0001988824 44.86005 149.54 0.9201889 0.0001993379 22.92012 0.600165 0.0001992147 0.0001987579 0.0001996667 0.08004232 105.1 0.0001994298 57.84015 16.82011 3.000137 0.0001985326 110.72 0.0001980098 2.000161 0.0001990028 141.6 0.0001987289 0.6201496 0.0001986333 0.0001987884 10.00019 157.62 
  - best initial criterion value(s) :  -7124.476 

N = 29, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=       7124.5  |proj g|=       5.4831
At iterate     1  f =         6975  |proj g|=     

[1] 1132375052 1132272313 1132383962 1112014713 1128295675 1130983076
 [7] 1130114378 1132167643 1122328756 1131584741 1130561370 1128813660
[13] 1131349322 1112534720 1118744344 1129979685 1124909052 1133591900
[19] 1132404943 1124241039 1124241039 1124542119 1124542119 1125174604
[25] 1123798007 1124165752 1125050687 1124970531 1126261933 1125655676
[31] 1125655676 1129162626 1080986569 1096891135 1085893391 1077001651
[37] 1095288102 1091024434 1095773194 1087139139 1107194789 1086904793
[43] 1108212313 1105844205 1099206580 1124165752 1122516913 1126618893
[49] 1125943106 1125943106

In [24]:
tmse <- function(d,mod, T)
{
    s = 0
    for(i in 1:1000){
    x = simul(d) 
    s = s + g(x, model = mod,T)
    }
    tmse = s/1000
    return(tmse)
}
    

In [25]:
print(tmse(d,mod,T=400))

[1] 1018315751


In [29]:
x = data_Lasso_reduced[1:50,]
shape = c(dim(data_reduced))
noise = matrix(runif(prod(shape),min = 0.0001,max = 0.0002), nrow = 120)
mod = km(formula = ~1, design = data_Lasso_reduced + noise, response = data_Lasso_reduced[1])




optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  476715169234 0.0001991599 44.86012 149.5402 0.9201991 0.0001981762 22.92016 0.6001607 0.0001988523 0.0001992747 0.0001989236 0.08006877 105.1002 0.000196887 57.84017 16.82005 3.000145 0.0001972013 110.7201 0.0001979035 2.000194 0.0001997556 141.6 0.0001991835 0.6200441 0.0001980778 0.0001992986 10.00018 157.6201 
  - best initial criterion value(s) :  -7159.889 

N = 29, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=       7159.9  |proj g|=      0.98367
At iterate     1  f =       6907.7  |proj

In [ ]:
new_tmse = function(x_new){
    
    
    
}

In [ ]:
X = data_Lasso_reduced
nrow = nrow(X)
for (i in 1:10){
    # Loop for individual search
    noise = matrix(runif(prod(dim(X)),min = 0.0001,max = 0.0002), nrow = nrow)
    mod = km(formula = ~1, design = X + noise, response = X[1])
    x_new = cobyla()
    
}